In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models, Input, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
edge_dataset_path = "processed_edges_dataset"  # Update to your actual folder
img_height, img_width = 224, 224
batch_size = 32

In [4]:
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    edge_dataset_path,
    target_size=(img_height, img_width),
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical',  # We will use one-hot encoded labels
    subset='training'
)

val_generator = datagen.flow_from_directory(
    edge_dataset_path,
    target_size=(img_height, img_width),
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

num_classes = train_generator.num_classes

Found 44369 images belonging to 39 classes.
Found 11077 images belonging to 39 classes.


In [5]:
edge_input = Input(shape=(img_height, img_width, 1), name="edge_input")

x = layers.Conv2D(32, (3, 3), activation='relu')(edge_input)
x = layers.MaxPooling2D((2, 2))(x)

x = layers.Conv2D(64, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)

x = layers.Conv2D(128, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)

x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu', name="edge_dense")(x)

edge_output = layers.Dense(num_classes, activation='softmax', name="edge_output")(x)

edge_cnn_model = Model(inputs=edge_input, outputs=edge_output, name="edge_cnn_model")

In [6]:
edge_cnn_model.compile(optimizer='adam',
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])

In [7]:
edge_cnn_model.summary()

Model: "edge_cnn_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ edge_input (InputLayer)         │ (None, 224, 224, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ edge_dense (Dense)              │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ edge_output (Dense)             │ (None, 39)             │         5,031 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,173,415 (42.62 MB)

 Trainable params: 11,173,415 (42.62 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
history = edge_cnn_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5
)


Epoch 1/5


C:\Users\aruno\anaconda3\envs\myenv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1387/1387 ━━━━━━━━━━━━━━━━━━━━ 585s 421ms/step - accuracy: 0.4632 - loss: 1.9948 - val_accuracy: 0.7497 - val_loss: 0.8269
Epoch 2/5
1387/1387 ━━━━━━━━━━━━━━━━━━━━ 570s 411ms/step - accuracy: 0.8297 - loss: 0.5494 - val_accuracy: 0.7896 - val_loss: 0.6742
Epoch 3/5
1387/1387 ━━━━━━━━━━━━━━━━━━━━ 569s 410ms/step - accuracy: 0.9494 - loss: 0.1648 - val_accuracy: 0.7821 - val_loss: 0.8074
Epoch 4/5
1387/1387 ━━━━━━━━━━━━━━━━━━━━ 569s 410ms/step - accuracy: 0.9769 - loss: 0.0734 - val_accuracy: 0.7798 - val_loss: 0.9931
Epoch 5/5
1387/1387 ━━━━━━━━━━━━━━━━━━━━ 566s 408ms/step - accuracy: 0.9813 - loss: 0.0562 - val_accuracy: 0.7778 - val_loss: 1.2684


In [9]:
edge_cnn_model.save("simple_cnn_edge_model.h5")